In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from MyNeuralNetwork import MyNeuralNetwork  # Import your custom neural network class
from torch.optim import lr_scheduler

# Define functions
def load_dataset(filename):
    try:
        df = pd.read_csv(filename)
        df = df.dropna(axis=1, how='all')  # Remove columns with all NaN values
        df.fillna(df.mean(), inplace=True)  # Fill NaN values with column means
        return df
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the file: {filename}. Error: {e}")
        return None

def clean_data(dataset):
    if not np.all(np.isfinite(dataset)):
        dataset = dataset.replace([np.inf, -np.inf], np.nan)
        dataset = dataset.dropna()
    return dataset

# Load and clean datasets
dataset1 = pd.read_csv('normalized_dataset1.csv')
dataset2 = pd.read_csv('normalized_dataset2.csv')
dataset3 = pd.read_csv('normalized_dataset3.csv')

dataset1 = clean_data(dataset1)
dataset2 = clean_data(dataset2)
dataset3 = clean_data(dataset3)

# Define input and output columns for each dataset
input_columns1 = ['height_over_sea_level', 'fall', 'net', 'fall_1']
output_column1 = 'flow'
input_columns2 = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']
output_column2 = 'z'
input_columns3 = ['X1 transaction date', 'X2 house age', 'X3 distance to the nearest MRT station',
                  'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']
output_column3 = 'Y house price of unit area'

# Normalize datasets using MinMaxScaler
scaler = MinMaxScaler()
X1 = scaler.fit_transform(dataset1[input_columns1])
y1 = scaler.fit_transform(dataset1[[output_column1]])
X2 = scaler.fit_transform(dataset2[input_columns2])
y2 = scaler.fit_transform(dataset2[[output_column2]])
X3 = scaler.fit_transform(dataset3[input_columns3])
y3 = scaler.fit_transform(dataset3[[output_column3]])

# Neural network configuration
num_units1 = [len(input_columns1), 10, 5, 1]
num_units2 = [len(input_columns2), 10, 5, 1]
num_units3 = [len(input_columns3), 10, 5, 1]

num_epochs = 500
learning_rate = 0.002
momentum = 0.9
activation_function = "relu"

# Split Dataset 1 into training and validation sets
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, test_size=0.15, random_state=42)

# Split Dataset 2 into training and validation sets
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.20, random_state=42)

# Split Dataset 3 into training and validation sets
X3_train, X3_val, y3_train, y3_val = train_test_split(X3, y3, test_size=0.20, random_state=42)

# Calculate the number of layers for each neural network
num_layers1 = len(num_units1)
num_layers2 = len(num_units2)
num_layers3 = len(num_units3)

# Initialize neural networks using your custom class
nn1 = MyNeuralNetwork(num_layers1, num_units1, num_epochs, learning_rate, momentum, activation_function , 0.15)
nn2 = MyNeuralNetwork(num_layers2, num_units2, num_epochs, learning_rate, momentum, activation_function , 0.20)
nn3 = MyNeuralNetwork(num_layers3, num_units3, num_epochs, learning_rate, momentum, activation_function , 0.20)

# Define the optimizer with weight decay for L2 regularization
optimizer1 = optim.Adam(nn1.parameters(), lr=learning_rate, weight_decay=1e-4)
optimizer2 = optim.Adam(nn2.parameters(), lr=learning_rate, weight_decay=1e-4)
optimizer3 = optim.Adam(nn3.parameters(), lr=learning_rate, weight_decay=1e-4)

# Define a learning rate scheduler for each optimizer
scheduler1 = lr_scheduler.StepLR(optimizer1, step_size=20, gamma=0.5)
scheduler2 = lr_scheduler.StepLR(optimizer2, step_size=20, gamma=0.5)
scheduler3 = lr_scheduler.StepLR(optimizer3, step_size=20, gamma=0.5)

# Inside your training loop, after each epoch, step the scheduler
scheduler1.step()
scheduler2.step()
scheduler3.step()

def calculate_mape(true, predicted):
    return (np.abs(true - predicted) / true) * 100


# Function to train the neural network and log training/validation loss
def train_and_log(nn, optimizer, X_train, y_train, X_val, y_val, num_epochs, batch_size=16):
    batch_size = int(batch_size)
    if batch_size <= 0:
        batch_size = 16
    
    training_loss = []
    validation_loss = []

    for epoch in range(num_epochs):
        # Training phase
        epoch_losses = []
        nn.train_model(X_train, y_train, num_epochs, learning_rate)
        for i in range(0, X_train.shape[0], batch_size):
            X_batch = torch.tensor(X_train[i:i+batch_size], dtype=torch.float32)
            y_batch = torch.tensor(y_train[i:i+batch_size], dtype=torch.float32)

            optimizer.zero_grad()
            outputs = nn(X_batch)
            loss = nn.calculate_loss(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
        avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        training_loss.append(avg_epoch_loss)

        # Validation phase
        nn.eval()
        with torch.no_grad():
            val_outputs = nn(torch.tensor(X_val, dtype=torch.float32))
            val_loss = nn.calculate_loss(val_outputs, torch.tensor(y_val, dtype=torch.float32))
            validation_loss.append(val_loss.item())

        print(f"Epoch {epoch}, Training Loss: {avg_epoch_loss}, Validation Loss: {validation_loss[-1]}")

    return training_loss, validation_loss

# Train the neural network for Dataset 1 and plot the training/validation loss
training_loss1, validation_loss1 = train_and_log(nn1, optimizer1, X1_train, y1_train, X1_val, y1_val, num_epochs, learning_rate)

# Plot training and validation loss for Dataset 1
plt.figure(figsize=(12, 6))
plt.plot(training_loss1, label='Training Loss')
plt.plot(validation_loss1, label='Validation Loss')
plt.title("Training and Validation Loss for Dataset 1")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Train the neural network for Dataset 2 and plot the training/validation loss
training_loss2, validation_loss2 = train_and_log(nn2, optimizer2, X2_train, y2_train, X2_val, y2_val, num_epochs, learning_rate)

# Plot training and validation loss for Dataset 2
plt.figure(figsize=(12, 6))
plt.plot(training_loss2, label='Training Loss')
plt.plot(validation_loss2, label='Validation Loss')
plt.title("Training and Validation Loss for Dataset 2")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Train the neural network for Dataset 3 and plot the training/validation loss
training_loss3, validation_loss3 = train_and_log(nn3, optimizer3, X3_train, y3_train, X3_val, y3_val, num_epochs, learning_rate)

# Plot training and validation loss for Dataset 3
plt.figure(figsize=(12, 6))
plt.plot(training_loss3, label='Training Loss')
plt.plot(validation_loss3, label='Validation Loss')
plt.title("Training and Validation Loss for Dataset 3")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# After training, calculate and print performance metrics for Dataset 1 validation set
# After training, calculate and print performance metrics for Dataset 1 validation set
nn1.eval()
with torch.no_grad():
    predicted1 = nn1(torch.tensor(X1_val, dtype=torch.float32)).numpy()
    true1 = y1_val.squeeze()
    mse1 = mean_squared_error(true1, predicted1)
    r2_1 = r2_score(true1, predicted1)
    mape1 = np.mean(np.abs(true1 - predicted1) / true1) * 100
print(f"Dataset 1 - MSE: {mse1:.2f}")
print(f"Dataset 1 - R2 Score: {r2_1:.2f}")
print(f"Dataset 1 - MAPE: {mape1:.2f}%")

# After training nn2 with Dataset 2, use X2_val and y2_val for validation
nn2.eval()
with torch.no_grad():
    predicted2 = nn2(torch.tensor(X2_val, dtype=torch.float32)).numpy()
    true2 = y2_val.squeeze()
    mse2 = mean_squared_error(true2, predicted2)
    r2_2 = r2_score(true2, predicted2)
    mape2 = np.mean(np.abs(true2 - predicted2) / true2) * 100
print(f"Dataset 2 - MSE: {mse2:.2f}")
print(f"Dataset 2 - R2 Score: {r2_2:.2f}")
print(f"Dataset 2 - MAPE: {mape2:.2f}%")

# After training nn3 with Dataset 3, use X3_val and y3_val for validation
nn3.eval()
with torch.no_grad():
    predicted3 = nn3(torch.tensor(X3_val, dtype=torch.float32)).numpy()
    true3 = y3_val.squeeze()
    mse3 = mean_squared_error(true3, predicted3)
    r2_3 = r2_score(true3, predicted3)
    mape3 = np.mean(np.abs(true3 - predicted3) / true3) * 100
print(f"Dataset 3 - MSE: {mse3:.2f}")
print(f"Dataset 3 - R2 Score: {r2_3:.2f}")
print(f"Dataset 3 - MAPE: {mape3:.2f}%")

/Users/hanyfaltaos/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/500, Loss: 0.08277950435876846
Epoch 2/500, Loss: 0.06156171609958013
Epoch 3/500, Loss: 0.05189103353768587
Epoch 4/500, Loss: 0.0471171032016476
Epoch 5/500, Loss: 0.04411729456235965
Epoch 6/500, Loss: 0.04047024389728904
Epoch 7/500, Loss: 0.03629425664742788
Epoch 8/500, Loss: 0.03159679612144828
Epoch 9/500, Loss: 0.0266985356186827
Epoch 10/500, Loss: 0.02187749200190107
Epoch 11/500, Loss: 0.016881242860108614
Epoch 12/500, Loss: 0.012456881192823252
Epoch 13/500, Loss: 0.00870639318600297
Epoch 14/500, Loss: 0.0057847892749123275
Epoch 15/500, Loss: 0.003885484708007425
Epoch 16/500, Loss: 0.002927655994426459
Epoch 17/500, Loss: 0.0022610077964297184
Epoch 18/500, Loss: 0.0020492796999557563
Epoch 19/500, Loss: 0.0019515727180987597
Epoch 20/500, Loss: 0.0019426040526013821
Epoch 21/500, Loss: 0.0019116016240635265
Epoch 22/500, Loss: 0.0018566859847245116
Epoch 23/500, Loss: 0.0018245203148884077
Epoch 24/500, Loss: 0.001812959594341616
Epoch 25/500, Loss: 0.00175222